##**Quantization of Mistral 7B**

In [ ]:
!pip install -U bitsandbytes # Install bitsandbytes

In [ ]:
!pip install transformers

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
from accelerate import init_empty_weights
from transformers import BitsAndBytesConfig

In [ ]:
import os
import sys
import psutil

# Function to measure memory usage (for CPU or GPU)
def get_memory_usage():
    # GPU Memory Usage
    if torch.cuda.is_available():
        return f"{torch.cuda.memory_allocated() / (1024 ** 3):.2f} GB (GPU)"
    # CPU Memory Usage
    process = psutil.Process(os.getpid())
    return f"{process.memory_info().rss / (1024 ** 3):.2f} GB (CPU)"

In [ ]:
MODEL_NAME = "mistralai/Ministral-8B-Instruct-2410"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
)

In [ ]:
print("Memory after loading model:", get_memory_usage())

Memory after loading model: 0.00 GB (GPU)


In [ ]:
model = AutoModelForCausalLM.from_pretrained(
        MODEL_NAME,
        quantization_config=bnb_config,
        torch_dtype=torch.bfloat16,
        device_map="auto",
        trust_remote_code=True,
    )

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [ ]:
print("Memory after loading model:", get_memory_usage())

Memory after loading model: 5.34 GB (GPU)


In [ ]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama_fast.LlamaTokenizerFast'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565 - if you loaded a llama tokenizer from a GGUF file you can ignore this message.


In [ ]:
from transformers import pipeline

In [ ]:
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer = tokenizer,
    torch_dtype=torch.bfloat16,
    device_map="auto"
)

In [ ]:
print("Memory after loading model:", get_memory_usage())

Memory after loading model: 5.34 GB (GPU)


In [ ]:
INSTRUCTION_KEY = "### Instruction:"
RESPONSE_KEY = "### Response:"
END_KEY = "### End"
INTRO_BLURB = "Below is an instruction that describes a task. Write a response that appropriately completes the request."
PROMPT_FOR_GENERATION_FORMAT = """{intro}
{instruction_key}
{instruction}
{end_key}
{response_key}
""".format(
    intro=INTRO_BLURB,
    instruction_key=INSTRUCTION_KEY,
    instruction="{instruction}",
    end_key=END_KEY,
    response_key=RESPONSE_KEY,
)

In [ ]:
def process_stream(instruction, temperature, top_p, top_k, max_new_tokens):
    #prompt = "As a data scientist, can you explain the concept of regularization in machine learning?"
    prompt = PROMPT_FOR_GENERATION_FORMAT.format(instruction=instruction.lower())

    sequences = pipe(
        prompt,
        do_sample=True,
        max_new_tokens= max_new_tokens, # 100,
        temperature=temperature, # 0.7,
        top_k=top_k, #50,
        top_p=top_p, #0.95,
        num_return_sequences=1,
    )
    return sequences[0]['generated_text']

In [ ]:
response = process_stream(instruction="What is the capital of India",top_p=50,top_k=1,temperature=1,max_new_tokens=100)
#response = generate_text(prompt_settings.instruction)
response = response.split('### Response:')[-1]
wrapped_text = response #textwrap.fill(response, width=100)

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


In [ ]:
print(wrapped_text)


The capital of India is New Delhi.


In [ ]:
print(response)


The capital of India is New Delhi.


##**Quantization of Mistral 8B**

In [ ]:
!pip install -U bitsandbytes # Install bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 5.2 MB/s eta 0:00:00


In [ ]:
!pip install transformers

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
from accelerate import init_empty_weights
from transformers import BitsAndBytesConfig

In [ ]:
import os
import sys
import psutil

# Function to measure memory usage (for CPU or GPU)
def get_memory_usage():
    # GPU Memory Usage
    if torch.cuda.is_available():
        return f"{torch.cuda.memory_allocated() / (1024 ** 3):.2f} GB (GPU)"
    # CPU Memory Usage
    process = psutil.Process(os.getpid())
    return f"{process.memory_info().rss / (1024 ** 3):.2f} GB (CPU)"

In [ ]:
MODEL_NAME = "mistralai/Mistral-7B-Instruct-v0.2"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
)

In [ ]:
print("Memory after loading model:", get_memory_usage())

Memory after loading model: 0.00 GB (GPU)


In [ ]:
model = AutoModelForCausalLM.from_pretrained(
        MODEL_NAME,
        quantization_config=bnb_config,
        torch_dtype=torch.bfloat16,
        device_map="auto",
        trust_remote_code=True,
    )

config.json:   0%|          | 0.00/596 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

In [ ]:
print("Memory after loading model:", get_memory_usage())

Memory after loading model: 3.84 GB (GPU)


In [ ]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/tokenization_auto.py:809: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.10k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [ ]:
from transformers import pipeline

In [ ]:
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer = tokenizer,
    torch_dtype=torch.bfloat16,
    device_map="auto"
)

In [ ]:
print("Memory after loading model:", get_memory_usage())

Memory after loading model: 3.85 GB (GPU)


In [ ]:
INSTRUCTION_KEY = "### Instruction:"
RESPONSE_KEY = "### Response:"
END_KEY = "### End"
INTRO_BLURB = "Below is an instruction that describes a task. Write a response that appropriately completes the request."
PROMPT_FOR_GENERATION_FORMAT = """{intro}
{instruction_key}
{instruction}
{end_key}
{response_key}
""".format(
    intro=INTRO_BLURB,
    instruction_key=INSTRUCTION_KEY,
    instruction="{instruction}",
    end_key=END_KEY,
    response_key=RESPONSE_KEY,
)

In [ ]:
def process_stream(instruction, temperature, top_p, top_k, max_new_tokens):
    #prompt = "As a data scientist, can you explain the concept of regularization in machine learning?"
    prompt = PROMPT_FOR_GENERATION_FORMAT.format(instruction=instruction.lower())

    sequences = pipe(
        prompt,
        do_sample=True,
        max_new_tokens= max_new_tokens, # 100,
        temperature=temperature, # 0.7,
        top_k=top_k, #50,
        top_p=top_p, #0.95,
        num_return_sequences=1,
    )
    return sequences[0]['generated_text']

In [ ]:
response = process_stream(instruction="What is the capital of India",top_p=50,top_k=1,temperature=1,max_new_tokens=100)
#response = generate_text(prompt_settings.instruction)
response = response.split('### Response:')[-1]
wrapped_text = response #textwrap.fill(response, width=100)

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


In [ ]:
print(wrapped_text)


The capital city of India is New Delhi. It is a city that holds great historical significance and is home to several iconic landmarks such as the Red Fort, Qutub Minar, and India Gate. New Delhi serves as the political and administrative hub of the country.


In [ ]:
print(response)


The capital city of India is New Delhi. It is a city that holds great historical significance and is home to several iconic landmarks such as the Red Fort, Qutub Minar, and India Gate. New Delhi serves as the political and administrative hub of the country.
